In [19]:
import os
import json

import numpy as np
import pandas as pd

from openai import OpenAI

from tqdm.auto import tqdm


SAMPLE_DATASET_CSV_FILE = "sample_dataset.csv"
DATASET_CSV_FILE = "dataset.csv"

In [20]:
from dotenv import load_dotenv

if os.path.isfile(".env"):
    load_dotenv()


In [21]:
def processing_dataset(
    df_book_data: pd.DataFrame,
    df_book_ratings: pd.DataFrame) -> pd.DataFrame:
    # remove unuseful columns from ratingd
    COLUMNS_DATASET = {
        "Title": "title",
        "review/score": "review_score",
        "review/summary": "review_summary",
        "review/text": "review_text"
    }
    COLUMN_LIST = list(COLUMNS_DATASET.keys())
    df_book_ratings = df_book_ratings[COLUMN_LIST]
    df_book_ratings = df_book_ratings.rename(columns=COLUMNS_DATASET)
    # remove unuseful columns from books data
    COLUMNS_DATASET = [
        "Title",
        "description",
        "authors",
        "publisher",
        "categories",
    ]
    df_book_data = df_book_data[COLUMNS_DATASET]
    df_book_data = df_book_data.rename(columns={"Title": "title"})
    # merge
    df_merged = pd.merge(left=df_book_ratings, right=df_book_data, on='title')
    df_merged = df_merged.fillna('')
    return df_merged


In [22]:
from dotenv import load_dotenv

if os.path.isfile(".env"):
    load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)


SAMPLE = os.getenv("SAMPLE")
if SAMPLE == "":
    SAMPLE = None

In [23]:
prompt_template = """
You emulate a review book assitant.
Formulate 5 questions this review book assitant might ask based on a FAQ record. The record
should contain the title of book, the book description, the author and short review, 
and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

title: {title}
authors: {authors}
categories: {categories}
summary: {review_summary}
description: {description}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()



def generate_questions(client, doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [24]:
if not os.path.isfile(SAMPLE_DATASET_CSV_FILE):

    if not os.path.isfile(DATASET_CSV_FILE):
        csv_file_1 = './ratings.csv'
        csv_file_2 = './data.csv'

        df1 = pd.read_csv(csv_file_1)
        df2 = pd.read_csv(csv_file_2)

        df = processing_dataset(df2, df1)
        df.to_csv(DATASET_CSV_FILE, index=False)

        if SAMPLE is not None:
            nsample = int(SAMPLE)
            df = df.sample(nsample,random_state=1)
    
    df = pd.read_csv(DATASET_CSV_FILE)

    sample_df = df.sample(200)
    sample_df.to_csv(SAMPLE_DATASET_CSV_FILE,index=False)

In [25]:
sample_df = pd.read_csv(SAMPLE_DATASET_CSV_FILE)

In [26]:
sample_df.head()

,title,review_score,review_summary,review_text,description,authors,publisher,categories
0,13 Cent Killers: The 5th Marine Snipers in Vie...,1.0,"Poorly written, bitter, amateurish",John Culbertson is a novice writer who is so d...,"“It’s not easy to stay alive with a $1,000 bou...",['John Culbertson'],Presidio Press,['History']
1,"One Shot (Jack Reacher, No. 9)",5.0,A page turner,"GrGreat story. As usual, great writing. Keeps ...",Six shots. Five dead. A heartland city thrown ...,['Lee Child'],Random House,['Fiction']
2,Jane Eyre,3.0,Had to do it.,Have always wanted to know what this was about...,NaN,['Charlotte Brontë'],NaN,['Literary Criticism']
3,Pleasuring The Prince,5.0,excellent Regency paranormal romantic suspense,In 1821 Fancy Flambeau debuts at London's Roya...,NaN,['Patricia Grasso'],Zebra Books,['Fiction']
4,Morbid Taste for Bones,3.0,Somewhat Contrived Ending,This is the first of a series and my first exp...,The “irresistible” and “compelling” first nove...,['Ellis Peters'],Open Road Media,['Fiction']


In [27]:
import hashlib

def generate_document_id(doc):
    combined = json.dumps(doc, sort_keys=True)
    hash_object = hashlib.md5(combined.encode(), usedforsecurity=False)
    hash_hex = hash_object.hexdigest()
    return hash_hex

In [28]:
documents = sample_df.to_dict(orient='records')
for i, doc in enumerate(documents):
    documents[i]['document_id'] = generate_document_id(doc)

In [29]:
df_documents = pd.DataFrame.from_records(documents)
df_documents.to_csv("df_documents.csv", index=False)

In [30]:
import json
import gzip

def save_jsongz(json_data, json_filepath):
    # Save JSON data to a .json.gz file
    with gzip.open(json_filepath, "wt", encoding="utf-8") as gz_file:
        json.dump(json_data, gz_file)

def read_jsongz(filepath):
    with gzip.open(filepath, "rt", encoding="utf-8") as gz_file:
        data = json.load(gz_file)
    return data


def save_jsongz_bin(json_data, json_filepath):
    with gzip.open(json_filepath, "wb") as gz_file:
        gz_file.write(json.dumps(json_data).encode("utf-8"))


def read_jsongz_bin(json_filepath):
    # Read JSON data from a binary .json.gz file
    with gzip.open(json_filepath, "rb") as gz_file:
        file_content = gz_file.read()  # Read the binary content
        data = json.loads(file_content.decode("utf-8"))  # Decode and load JSON
    return data


In [31]:
RESULTS_JSON_FILEPATH = "results.json.gz"


if not  os.path.isfile(RESULTS_JSON_FILEPATH):
    results = {}
    for doc in tqdm(documents): 
        doc_id = doc['document_id']
        if doc_id in results:
            continue
        questions = generate_questions(openai_client, doc)
        results[doc_id] = questions
    save_jsongz_bin(results, RESULTS_JSON_FILEPATH)


results = read_jsongz_bin(RESULTS_JSON_FILEPATH)

  0%|          | 0/200 [00:00<?, ?it/s]

In [32]:

ground_truth = []
for rst_key, rst_value in results.items():
    questions = json.loads(rst_value)
    df_temp = df_documents[df_documents["document_id"] == rst_key]
    if df_temp.size == 0:
        continue
    for q in questions:
        item = {}
        item["document_id"] = rst_key
        item["question"] = q
        item["title"] = df_temp.iloc[0]["title"]
        item["authors"] = df_temp.iloc[0]["authors"]
        item["title"] = df_temp.iloc[0]["title"]
        item["categories"] = df_temp.iloc[0]["categories"]
        ground_truth.append(item)


In [33]:
df_ground_truth = pd.DataFrame.from_records(ground_truth)
df_ground_truth.to_csv("df_ground_truth.csv", index=False)

In [34]:
df_ground_truth = pd.read_csv("df_ground_truth.csv")

In [35]:
df_ground_truth.head()

,document_id,question,title,authors,categories
0,8bfa3b6f784445636db49a39649fd346,What makes the 5th Marine Regiment Sniper Plat...,13 Cent Killers: The 5th Marine Snipers in Vie...,['John Culbertson'],['History']
1,8bfa3b6f784445636db49a39649fd346,What role did sniper teams play alongside Mari...,13 Cent Killers: The 5th Marine Snipers in Vie...,['John Culbertson'],['History']
2,8bfa3b6f784445636db49a39649fd346,"How does the author, John J. Culbertson, descr...",13 Cent Killers: The 5th Marine Snipers in Vie...,['John Culbertson'],['History']
3,8bfa3b6f784445636db49a39649fd346,What are some of the challenges faced by the 5...,13 Cent Killers: The 5th Marine Snipers in Vie...,['John Culbertson'],['History']
4,8bfa3b6f784445636db49a39649fd346,How does the book highlight the sacrifices mad...,13 Cent Killers: The 5th Marine Snipers in Vie...,['John Culbertson'],['History']
